# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %reload_ext autoreload
# import gc
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)
# torch.cuda.empty_cache()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os

# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
# import warnings
# warnings.filterwarnings('ignore')
# C:/Users/sshar/AppData/Roaming/jupyter/nbextensions/snippets /snippets.json (jupyter --data-dir)

In [21]:
from dataset import CheXpertDataset
import utils
from utils import vprint
from utils import to_gpu

# Configs 

In [4]:
@dataclass
class TrainingConfigs:
    DATA_DIR = os.path.join("..", "data", "CheXpert", "CheXpert-v1.0-small")
    CHECKPOINT_DIR = r"checkpoints"
    BATCH_SIZE = 4
    EPOCHS = 10
    LEARNING_RATE = 0.0001
    CHECKPOINT_TIME_INTERVAL = 5 # seconds
    MODEL_VERSION = "densenet121"
    TRAINED_MODEL_PATH = None
    TRAIN_LOADER_SIZE = None
    VALID_LOADER_SIZE = None
    VALID_SIZE = 12 # for debugging purposes

In [5]:
utils.set_seed()

# Training

## Training Setup

In [6]:
train_transform = transforms.Compose([
#     transforms.Resize((320,320)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     # augmentation 
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.01)], p=0.1),
    transforms.RandomApply([torchvision.transforms.GaussianBlur(kernel_size=(3,3) ,sigma=(0.25, 0.75))], p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=0.75, p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=1.25, p=0.1)
])

valid_transform = transforms.Compose([
#     transforms.Resize((320,320)),
    transforms.Resize((224,224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
# Create data loaders.
train_dataset = CheXpertDataset(labels_filename='train.csv', data_dir=TrainingConfigs.DATA_DIR, transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=True)
TrainingConfigs.TRAIN_LOADER_SIZE = len(train_dataloader)
len(train_dataset)

223414

In [8]:
valid_dataset = CheXpertDataset(labels_filename='valid.csv', data_dir=TrainingConfigs.DATA_DIR, transform=valid_transform)
valid_dataset.labels = valid_dataset.labels[:TrainingConfigs.VALID_SIZE] # hack for speed debugging
valid_dataloader = DataLoader(valid_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.VALID_LOADER_SIZE = len(valid_dataloader)
len(valid_dataset)

12

In [9]:
# model = densenet161(weights=DenseNet161_Weights.DEFAULT)
# num_features = model.classifier.in_features
# model.classifier = nn.Sequential(
#     nn.Linear(num_features, num_features, bias=True),
#     nn.ReLU(),
#     nn.Dropout(p=0.1),
#     nn.Linear(in_features=num_features, out_features=utils.Configs.NUM_CLASSES, bias=True)
# )

In [10]:
model = densenet121(weights=DenseNet121_Weights.DEFAULT)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=utils.Configs.NUM_CLASSES, bias=True)
)

In [11]:
# model = torchvision.models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
# num_features = model.heads.head.in_features
# model.heads.head = nn.Sequential(
#     nn.Linear(num_features, num_features, bias=True),
#     nn.ReLU(),
#     nn.Dropout(p=0.1),
#     nn.Linear(in_features=num_features, out_features=utils.Configs.NUM_CLASSES, bias=True)
# )

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=TrainingConfigs.LEARNING_RATE, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, mode='min')
criterion = nn.BCEWithLogitsLoss(reduction='mean') # combines BCEntropy and sigmoid
# final nn labels: torch.round(torch.sigmoid(pred))
# simple solution to handle the multi label problem (probabilities don't have to sum to 1)

## Training Loop 

In [22]:
model, results, last_epoch, last_iter = utils.get_previous_training_place(model, TrainingConfigs)
model.train()
model = to_gpu(model)
start_time = time.time()
for epoch in range(last_epoch, TrainingConfigs.EPOCHS):
    train_dataloader_iter = islice(tqdm(enumerate(train_dataloader), total=len(train_dataloader)), 
                                   last_iter+1, len(train_dataloader)) # fast foward dataloader
    for i, (images, labels) in train_dataloader_iter:
        images = to_gpu(images)
        labels = to_gpu(labels)        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        results['train_loss'].append(loss.item())
        if time.time()-start_time > TrainingConfigs.CHECKPOINT_TIME_INTERVAL:
            utils.create_checkpoint(model, epoch, i, valid_dataloader, criterion, results, TrainingConfigs,
                                    by_study='max', challenge_ann_only=True)
            start_time = time.time()
    scheduler.step(np.mean(results["valid_loss"][-len(train_dataloader):]))

2022-07-10 18:39: Loading model - checkpoints\2022_07_10-18_34__densenet121__epoch-0__iter-9__batch_size-4__trainLastLoss-0.3771__validAUC-0.4966.dict


  0%|          | 0/55854 [00:00<?, ?it/s]

2022-07-10 18:39: 2022_07_10-18_39: Checkpoint Created.
2022-07-10 18:39: Epoch [1/10],   Iter [11/55853],   Train Loss: 0.3850,   Valid Loss: 1.2435,   Valid AUC: 0.3166

2022-07-10 18:39: 2022_07_10-18_39: Checkpoint Created.
2022-07-10 18:39: Epoch [1/10],   Iter [13/55853],   Train Loss: 0.3601,   Valid Loss: 1.4649,   Valid AUC: 0.2246

2022-07-10 18:39: 2022_07_10-18_39: Checkpoint Created.
2022-07-10 18:39: Epoch [1/10],   Iter [15/55853],   Train Loss: 0.3698,   Valid Loss: 1.1359,   Valid AUC: 0.5565

2022-07-10 18:39: 2022_07_10-18_39: Checkpoint Created.
2022-07-10 18:39: Epoch [1/10],   Iter [17/55853],   Train Loss: 0.3733,   Valid Loss: 1.1162,   Valid AUC: 0.6078

2022-07-10 18:39: 2022_07_10-18_39: Checkpoint Created.
2022-07-10 18:39: Epoch [1/10],   Iter [19/55853],   Train Loss: 0.3544,   Valid Loss: 1.3271,   Valid AUC: 0.6406

2022-07-10 18:40: 2022_07_10-18_40: Checkpoint Created.
2022-07-10 18:40: Epoch [1/10],   Iter [21/55853],   Train Loss: 0.3627,   Valid Los

KeyboardInterrupt: 